In [251]:
#Import Libraries Needed
import pandas as pd
import geopandas as gpd
import panel as pn
pn.extension('plotly',comms='ipywidgets')
import param
import hvplot.pandas
import plotly.express as px
from urllib.request import urlopen
import plotly.graph_objects as go
import requests
from PIL import Image, ImageOps

In [252]:
css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
  font-size: 24px;
}
'''

pn.extension(raw_css=[css])

In [253]:
#Import DataFrames
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
target_states = ['48']
counties['features'] = [f for f in counties['features'] if f['properties']['STATE'] in target_states]
address = ("Data/District_Loss/district18-22.csv")
dis_df = pd.read_csv(address)

#Create a new dataframe with columns based off the differences of 2021 National Center on Education Statistics and
#2019 National Center on Education Statistics, divided by the 2019 data.
dis_copy = dis_df.copy()
countynames = dis_df['County Name'].unique()
countynames.sort()
fips = dis_df['County #'].unique()
fips.sort()
dis_copy = dis_copy.groupby('County Name').mean()
dis_copy = dis_copy.drop(['District #'], axis = 1)
dis_copy['County #'] = fips
dis_copy['County Name'] = countynames
dis_copy= dis_copy.reset_index(drop=True)

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2021-2019)'] = (dis_copy['FTE Teachers 2020-2021']-dis_copy['FTE Teachers 2018-2019'])/dis_copy['FTE Teachers 2018-2019']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)'] = (dis_copy['Total Staff 2020-2021']-dis_copy['Total Staff 2018-2019'])/dis_copy['Total Staff 2018-2019']
dis_copy['Total Number of Students Difference (2021-2019)'] = (dis_copy['Total Students 2020-2021']-dis_copy['Total Students 2018-2019'])/dis_copy['Total Students 2018-2019']

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2021-2022)'] = (dis_copy['FTE Teachers 2021-2022']-dis_copy['FTE Teachers 2020-2021'])/dis_copy['FTE Teachers 2020-2021']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)'] = (dis_copy['Total Staff 2021-2022']-dis_copy['Total Staff 2020-2021'])/dis_copy['Total Staff 2020-2021']
dis_copy['Total Number of Students Difference (2021-2022)'] = (dis_copy['Total Students 2021-2022']-dis_copy['Total Students 2020-2021'])/dis_copy['Total Students 2020-2021']

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2019-2022)'] = (dis_copy['FTE Teachers 2021-2022']-dis_copy['FTE Teachers 2018-2019'])/dis_copy['FTE Teachers 2018-2019']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)'] = (dis_copy['Total Staff 2021-2022']-dis_copy['Total Staff 2018-2019'])/dis_copy['Total Staff 2018-2019']
dis_copy['Total Number of Students Difference (2019-2022)'] = (dis_copy['Total Students 2021-2022']-dis_copy['Total Students 2018-2019'])/dis_copy['Total Students 2018-2019']

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\1790098922.py:17: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [255]:
dis_copy = dis_copy.round(2)

In [256]:
#Import dataframes
address = ("Data/District_Loss/txcountyemploy.xlsx")
employ_df = pd.read_excel(address)

#Create a new dataframes with columns based on the differences of the 2021 Bureau of Labor Statistics Data and 
#2019 Bureau of Labor Statistics Data, divided by the 2019 data.
employ_df['NAME'] = employ_df['NAME'].str.replace(", TX","")
employ_df['YEAR']=employ_df['YEAR'].values.astype(str)
employ_df['YEAR'] = employ_df['YEAR'].str.replace("'","")
df_19 = employ_df[employ_df['YEAR'] == '2019']
df_20 = employ_df[employ_df['YEAR'] == '2020']
df_21 = employ_df[employ_df['YEAR'] == '2021']
df_20 =df_20.reset_index()
df_21=df_21.reset_index()
diff_df = pd.DataFrame()
diff_df = df_19[['NAME','FIPS']]
diff_df['Civilian Labor Force Difference (2021-2019)'] = (df_21['LABOR_FORCE']-df_19['LABOR_FORCE'])/df_19['LABOR_FORCE']
diff_df['Employed Persons Difference (2021-2019)']= (df_21['EMPLOYED']-df_19['EMPLOYED'])/df_19['EMPLOYED']
diff_df['Umemployed Persons Difference (2021-2019)']= (df_21['UNEMPLOYED']-df_19['UNEMPLOYED'])/df_19['UNEMPLOYED']
diff_df['Unemployment Rate Difference (2021-2019)']= (df_21['UNEMPLOYMENT_PCT']-df_19['UNEMPLOYMENT_PCT'])/df_19['UNEMPLOYMENT_PCT']

D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\1615083697.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\1615083697.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\1615083697.py:19: SettingWithCopyWarning:


A value is trying to be set on 

In [257]:
diff_df = diff_df.round(2)
diff_df.rename(columns={'NAME':'County Name','FIPS':'County #'},inplace=True)

In [258]:
#Import dataframes
address = ("Data/District_Loss/DATA_Texas_District_v1.csv")
disloss_df = pd.read_csv(address)
address = ("Data/District_Loss/district18-22.csv")
dis_df = pd.read_csv(address)

#Creates a dataframe based on the Texas Education Agency data based on the differences of
#2021 and 2019 data, divided by 2019 data.
disloss_df['County #'] = disloss_df['County #'].astype(int)
countynames = dis_df['County Name'].unique()
countynames.sort()
fips = dis_df['County #'].unique()
fips.sort()

discopy_df = pd.DataFrame()
discopy_df['County #'] = disloss_df['County #']
discopy_df['Average Score Difference(%) in Math (2021-2019)'] = (disloss_df['AVG_Math_2021']-disloss_df['AVG_Math_2019'])/disloss_df['AVG_Math_2019']
discopy_df['Average Score Difference(%) in Reading (2021-2019)'] = (disloss_df['AVG_Reading_2021']-disloss_df['AVG_Reading_2019'])/disloss_df['AVG_Reading_2019']
discopy_df['Average Score Difference(%) in Math (2022-2021)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2021'])/disloss_df['AVG_Math_2021']
discopy_df['Average Score Difference(%) in Reading (2022-2021)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2021'])/disloss_df['AVG_Reading_2021']
discopy_df['Average Score Difference(%) in Math (2022-2019)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2019'])/disloss_df['AVG_Math_2019']
discopy_df['Average Score Difference(%) in Reading (2022-2019)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2019'])/disloss_df['AVG_Reading_2019']

discopy_df = discopy_df.groupby('County #').mean()
discopy_df['County #'] = fips
discopy_df['County Name'] = countynames
discopy_df = discopy_df.reset_index(drop=True)

In [259]:
discopy_df = discopy_df.round(2)

In [260]:
discopy_df.drop(columns='County Name',inplace=True)
dis_copy.drop(columns='County Name',inplace=True)
comp_df = discopy_df.merge(diff_df,on='County #')
comp_df = comp_df.merge(dis_copy,on='County #')

In [261]:
comp_df.columns

Index(['Average Score Difference(%) in Math (2021-2019)',
       'Average Score Difference(%) in Reading (2021-2019)',
       'Average Score Difference(%) in Math (2022-2021)',
       'Average Score Difference(%) in Reading (2022-2021)',
       'Average Score Difference(%) in Math (2022-2019)',
       'Average Score Difference(%) in Reading (2022-2019)', 'County #',
       'County Name', 'Civilian Labor Force Difference (2021-2019)',
       'Employed Persons Difference (2021-2019)',
       'Umemployed Persons Difference (2021-2019)',
       'Unemployment Rate Difference (2021-2019)', 'FTE Teachers 2018-2019',
       'Total Staff 2018-2019', 'Total Students 2018-2019',
       'FTE Teachers 2020-2021', 'Total Staff 2020-2021',
       'Total Students 2020-2021', 'FTE Teachers 2021-2022',
       'Total Staff 2021-2022', 'Total Students 2021-2022',
       'Full-Time Equivalent(FTE) Teachers Difference (2021-2019)',
       'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (202

In [262]:
comp_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2021-2019)':'Number of Teachers (2019-2021)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)':'Number of Overall Staff (2019-2021)',
                          'Total Number of Students Difference (2021-2019)': 'Number of Students (2019-2021)',
                          'Civilian Labor Force Difference (2021-2019)':'Labor Force (2019-2021)',
    
                          'Average Score Difference(%) in Math (2021-2019)': 'STAAR Score in Math (2019-2021)',
                          'Average Score Difference(%) in Reading (2021-2019)': 'STAAR Score in Reading (2019-2021)',
                          'Average Score Difference(%) in Math (2022-2021)': 'STAAR Score in Math (2021-2022)',
                          'Average Score Difference(%) in Reading (2022-2021)': 'STAAR Score in Reading (2021-2022)',
                          'Average Score Difference(%) in Math (2022-2019)': 'STAAR Score in Math (2019-2022)',
                          'Average Score Difference(%) in Reading (2022-2019)': 'STAAR Score in Reading (2019-2022)',
                        
                          'Full-Time Equivalent(FTE) Teachers Difference (2021-2022)':'Number of Teachers (2021-2022)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)': 'Number of Overall Staff (2021-2022)',
                          'Total Number of Students Difference (2021-2022)': 'Number of Students (2021-2022)',
                        
                          'Full-Time Equivalent(FTE) Teachers Difference (2019-2022)':'Number of Teachers (2019-2022)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)': 'Number of Overall Staff (2019-2022)',
                          'Total Number of Students Difference (2019-2022)': 'Number of Students (2019-2022)'},
                          inplace=True)

In [264]:
comp_df['Number of Teachers (2019-2021)'] = comp_df['Number of Teachers (2019-2021)']*100
comp_df['Number of Overall Staff (2019-2021)'] = comp_df['Number of Overall Staff (2019-2021)']*100
comp_df['Number of Students (2019-2021)'] = comp_df['Number of Students (2019-2021)']*100

comp_df['Number of Teachers (2021-2022)'] = comp_df['Number of Teachers (2021-2022)']*100
comp_df['Number of Overall Staff (2021-2022)'] = comp_df['Number of Overall Staff (2021-2022)']*100
comp_df['Number of Students (2021-2022)'] = comp_df['Number of Students (2021-2022)']*100

comp_df['Number of Teachers (2019-2022)'] = comp_df['Number of Teachers (2019-2022)']*100
comp_df['Number of Overall Staff (2019-2022)'] = comp_df['Number of Overall Staff (2019-2022)']*100
comp_df['Number of Students (2019-2022)'] = comp_df['Number of Students (2019-2022)']*100

comp_df['STAAR Score in Math (2019-2021)'] = comp_df['STAAR Score in Math (2019-2021)']*100
comp_df['STAAR Score in Reading (2019-2021)'] = comp_df['STAAR Score in Reading (2019-2021)']*100
comp_df['STAAR Score in Math (2021-2022)'] = comp_df['STAAR Score in Math (2021-2022)']*100
comp_df['STAAR Score in Reading (2021-2022)'] = comp_df['STAAR Score in Reading (2021-2022)']*100
comp_df['STAAR Score in Math (2019-2022)'] = comp_df['STAAR Score in Math (2019-2022)']*100
comp_df['STAAR Score in Reading (2019-2022)'] = comp_df['STAAR Score in Reading (2019-2022)']*100

comp_df['Labor Force (2019-2021)'] = comp_df['Labor Force (2019-2021)']*100
comp_df['County Name'] = comp_df['County Name'].str[:-7]

In [265]:
comp_df.dtypes

STAAR Score in Math (2019-2021)              float64
STAAR Score in Reading (2019-2021)           float64
STAAR Score in Math (2021-2022)              float64
STAAR Score in Reading (2021-2022)           float64
STAAR Score in Math (2019-2022)              float64
STAAR Score in Reading (2019-2022)           float64
County #                                       int64
County Name                                   object
Labor Force (2019-2021)                      float64
Employed Persons Difference (2021-2019)      float64
Umemployed Persons Difference (2021-2019)    float64
Unemployment Rate Difference (2021-2019)     float64
FTE Teachers 2018-2019                       float64
Total Staff 2018-2019                        float64
Total Students 2018-2019                     float64
FTE Teachers 2020-2021                       float64
Total Staff 2020-2021                        float64
Total Students 2020-2021                     float64
FTE Teachers 2021-2022                       f

# Texas County Level Map

In [266]:
comp_df['STAAR Score in Math (2021-2022)'] = comp_df['STAAR Score in Math (2021-2022)'].astype(int)
comp_df['STAAR Score in Reading (2021-2022)'] = comp_df['STAAR Score in Reading (2021-2022)'].astype(int)

comp_df['STAAR Score in Math (2019-2022)'] = comp_df['STAAR Score in Math (2019-2022)'].astype(int)
comp_df['STAAR Score in Reading (2019-2022)'] = comp_df['STAAR Score in Reading (2019-2022)'].astype(int)

comp_df['STAAR Score in Math (2019-2021)'] = comp_df['STAAR Score in Math (2019-2021)'].astype(int)
comp_df['STAAR Score in Reading (2019-2021)'] = comp_df['STAAR Score in Reading (2019-2021)'].astype(int)
comp_df['Labor Force (2019-2021)'] = comp_df['Labor Force (2019-2021)'].astype(int)


comp_df['Number of Teachers (2021-2022)'] = comp_df['Number of Teachers (2021-2022)'].astype(int)
comp_df['Number of Overall Staff (2021-2022)'] = comp_df['Number of Overall Staff (2021-2022)'].astype(int)
comp_df['Number of Students (2021-2022)'] = comp_df['Number of Students (2021-2022)'].astype(int)

comp_df['Number of Teachers (2019-2022)'] = comp_df['Number of Teachers (2019-2022)'].astype(int)
comp_df['Number of Overall Staff (2019-2022)'] = comp_df['Number of Overall Staff (2019-2022)'].astype(int)
comp_df['Number of Students (2019-2022)'] = comp_df['Number of Students (2019-2022)'].astype(int)

comp_df['Number of Teachers (2019-2021)'] = comp_df['Number of Teachers (2019-2021)'].astype(int)
comp_df['Number of Overall Staff (2019-2021)'] = comp_df['Number of Overall Staff (2019-2021)'].astype(int)
comp_df['Number of Students (2019-2021)'] = comp_df['Number of Students (2019-2021)'].astype(int)

In [268]:
comp_df.to_csv('Data/CountyLevelstats.csv')

In [269]:
df1921 = comp_df[['County Name','County #','Number of Teachers (2019-2021)',
                  'Number of Overall Staff (2019-2021)',
                  'Number of Students (2019-2021)',
                  'Labor Force (2019-2021)',
                  'STAAR Score in Math (2019-2021)',
                  'STAAR Score in Reading (2019-2021)']]

df2122 = comp_df[['County Name','County #','Number of Teachers (2021-2022)',
                  'Number of Overall Staff (2021-2022)',
                  'Number of Students (2021-2022)',
                  'STAAR Score in Math (2021-2022)',
                  'STAAR Score in Reading (2021-2022)']]

df1922 = comp_df[['County Name','County #','Number of Teachers (2019-2022)',
                  'Number of Overall Staff (2019-2022)',
                  'Number of Students (2019-2022)',
                  'STAAR Score in Math (2019-2022)',
                  'STAAR Score in Reading (2019-2022)']]

In [272]:
df1921.rename(columns = {'Number of Teachers (2019-2021)':'Number of Teachers',
                           'Number of Overall Staff (2019-2021)':'Number of Staff',
                           'Number of Students (2019-2021)':'Number of Students',
                           'STAAR Score in Math (2019-2021)':'STAAR Score in Math',
                           'STAAR Score in Reading (2019-2021)':'STAAR Score in Reading'
                           }, inplace=True)

df2122.rename(columns = {'Number of Teachers (2021-2022)':'Number of Teachers',
                           'Number of Overall Staff (2021-2022)':'Number of Staff',
                           'Number of Students (2021-2022)':'Number of Students',
                           'STAAR Score in Math (2021-2022)':'STAAR Score in Math',
                           'STAAR Score in Reading (2021-2022)':'STAAR Score in Reading',
                           }, inplace=True)

df1922.rename(columns = {'Number of Teachers (2019-2022)':'Number of Teachers',
                           'Number of Overall Staff (2019-2022)':'Number of Staff',
                           'Number of Students (2019-2022)':'Number of Students',
                           'STAAR Score in Math (2019-2022)':'STAAR Score in Math',
                           'STAAR Score in Reading (2019-2022)':'STAAR Score in Reading',
                           }, inplace=True)

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\3827061595.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\3827061595.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\3827061595.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [307]:
#Creates a county level map of Texas based on the Texas Education Agency Data
input_list = ['STAAR Score in Math',
              'STAAR Score in Reading',
              'Number of Students',
              'Number of Teachers',
              'Number of Staff']
# Labor Force Taken out for now 2019-2021
year_list = ['2019-2021','2021-2022','2019-2022']
class MapDashboard(param.Parameterized):
    _  = param.Selector(input_list,default=input_list[0])
    __ = param.Selector(default=year_list[0],objects=year_list)
    #@param.depends('_')
    def get_map(self):
        if self.__ == '2019-2021':
            data = df1921[['County Name','County #',self._]]
        elif self.__ == '2021-2022':
            data = df2122[['County Name','County #',self._]]
        elif self.__ == '2019-2022':
            data = df1922[['County Name','County #',self._]]
        fig = px.choropleth_mapbox(data, geojson=counties, locations='County #', color= self._,
        color_continuous_scale='balance_r',
        range_color=(-10,10),
        center = {'lat':31.9686, 'lon': -99.9018},
        zoom = 4.65,
        height=585, width=525,
        mapbox_style = "white-bg",       
        custom_data = [data['County Name'],self._]
        )
        hovertemp = ''
        hovertemp = '%{customdata[0]} County<br>'                   
        hovertemp += '%{customdata[1]}% change in ' + self._
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations')
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=24,color='black'),                
        margin={"r":10,"t":0,"l":10,"b":0})
        fig.update_layout(coloraxis_colorbar=dict(tickvals=[-8,-4,0,4,8],ticktext=['-10','-5','0','5','10'])),
        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-.148,colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=20,colorbar_title_side='bottom', colorbar_thickness=4)
        return fig
    def get_description(self):
        if self.__ == '2019-2021':
            if self._ == 'STAAR Score in Math':
                description = pn.pane.HTML('The percent difference in Math STAAR scores between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'STAAR Score in Reading':
                description = pn.pane.HTML('The percent difference in Reading STAAR scores between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Labor Force':
                description = pn.pane.HTML('The percent difference in labor force between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Teachers':
                description = pn.pane.HTML('The percent difference in Full-Time Teachers between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Staff':
                description = pn.pane.HTML('The percent difference in staff between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._== 'Number of Students':
                description = pn.pane.HTML('The percent difference in students between 2019 and 2021 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self.__ == '2021-2022':
            if self._ == 'STAAR Score in Math':
                description = pn.pane.HTML('The percent difference in Math STAAR scores between 2021 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'STAAR Score in Reading':
                description = pn.pane.HTML('The percent difference in Reading STAAR scores between 2021 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Labor Force':
                description = pn.pane.HTML('The percent difference in labor force between 2021 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Teachers':
                description = pn.pane.HTML('The percent difference in Full-Time Teachers between 2021 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Staff':
                description = pn.pane.HTML('The percent difference in staff between 2021 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._== 'Number of Students':
                description = pn.pane.HTML('The percent difference in students between 2021 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self.__ == '2019-2022':
            if self._ == 'STAAR Score in Math':
                description = pn.pane.HTML('The percent difference in Math STAAR scores between 2019 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'STAAR Score in Reading':
                description = pn.pane.HTML('The percent difference in Reading STAAR scores between 2019 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Labor Force':
                description = pn.pane.HTML('The percent difference in labor force between 2019 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Teachers':
                description = pn.pane.HTML('The percent difference in Full-Time Teachers between 2019 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Staff':
                description = pn.pane.HTML('The percent difference in staff between 2019 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._== 'Number of Students':
                description = pn.pane.HTML('The percent difference in students between 2019 and 2022 for a county.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        return description
    def panel(self):
        return pn.Row(
            pn.Column(
            pn.pane.HTML("""<h1>Select A Variable To View At The County Level</h1>""",
                              style={'color':'black','font-size':'14px','width':'100%',
                                     'text-align':'center',
                                     'font-family':'Times New Roman'}),
            pn.Param(self,width=550,background='#FFFFFF',
            widgets={'__':{'widget_type':pn.widgets.RadioButtonGroup}}),
            pn.Spacer(height=50),
            pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            style={'color':'black','font-size':'14px','width':'100%',
                                  'text-align':'center',
                                  'font-family':'Times New Roman'}),
            pn.Spacer(height= 5),
            self.get_description
            ),
            pn.Spacer(width=150),
            self.get_map
        )
dashboard = MapDashboard(name='')
map_component1 = dashboard.panel()
map_component1

BokehModel(combine_events=True, render_bundle={'docs_json': {'678641b5-061e-490a-adbd-51eca07518b2': {'defs': …

# Heatmap & Histogram

In [274]:
#Import dataframe
address = ("Data/District_Loss/DATA_Texas_District_v1.csv")
loss_df = pd.read_csv(address)
loss_df = loss_df.round(2)

In [275]:
loss_df['% LOSS Math 2019 to 2022'] = loss_df['% LOSS Math 2019 to 2022']*100
loss_df['% LOSS Reading 2019 to 2022'] = loss_df['% LOSS Reading 2019 to 2022']*100
loss_df['% LOSS Math 2021 to 2022'] = loss_df['% LOSS Math 2021 to 2022']*100
loss_df['% LOSS Reading 2021 to 2022'] = loss_df['% LOSS Reading 2021 to 2022']*100
loss_df['% LOSS Math 2019 to 2021'] = loss_df['% LOSS Math 2019 to 2021']*100
loss_df['% LOSS Reading 2019 to 2021'] = loss_df['% LOSS Reading 2019 to 2021']*100

In [278]:
#Creates a histogram based on the STAAR/Percent Loss data, displays percent loss by district count
input_list = ['2019-2021','2021-2022','2019-2022']
class LossDashboard(param.Parameterized):
    _ = param.Selector(input_list,default=input_list[0])
    #@param.depends('_')
    def get_map(self):
        if self._ == '2019-2021':
            #data = loss_df[['% LOSS Math 2019 to 2021','% LOSS Reading 2019 to 2021']]
            fig = go.Figure(data=[go.Histogram(x=loss_df['% LOSS Math 2019 to 2021'],
                                      name='Math')])
            fig.add_trace(go.Histogram(x=loss_df['% LOSS Reading 2019 to 2021'],
                                      name='Reading'))
            fig.update_layout(barmode='stack')
        elif self._ == '2021-2022':
            #data = loss_df[['% LOSS Math 2021 to 2022','% LOSS Reading 2021 to 2022']]
            fig = go.Figure(data=[go.Histogram(x=loss_df['% LOSS Math 2021 to 2022'],
                                      name='Math')])
            fig.add_trace(go.Histogram(x=loss_df['% LOSS Reading 2021 to 2022'],
                                      name='Reading'))
            fig.update_layout(barmode='stack')
        elif self._ == '2019-2022':
            #data = loss_df[['% LOSS Math 2019 to 2022','% LOSS Reading 2019 to 2022']]
            fig = go.Figure(data=[go.Histogram(x=loss_df['% LOSS Math 2019 to 2022'],
                                      name='Math')])
            fig.add_trace(go.Histogram(x=loss_df['% LOSS Reading 2019 to 2022'],
                                      name='Reading'))
            fig.update_layout(barmode='stack')
            
        fig.update_xaxes(range=[-30,30],dtick=10)
        fig.update_yaxes(range=[0,600],dtick=100)
       # fig.update_traces(opacity=0.7)
        fig.update_layout(height=540, width =680,
                          xaxis_title_text='% Loss', 
                          yaxis_title_text='Number of Districts',
                          bargap=0.2, hovermode=False,
                          hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=23,color='black'),
        title_x=0.55,
       # title_y=.95,
        margin={"r":10,"t":10,"l":0,"b":15})
        return fig
    def panel(self):
        return pn.Column(
            pn.Param(self,width=450,background='#FFFFFF',
                     widgets={'_':{'widget_type':pn.widgets.RadioButtonGroup}}),            
            self.get_map
        )
dashboard = LossDashboard(name = 'Normalized Difference in STAAR Scores, Avg. Over TX ISDs')
loss_component1 = dashboard.panel()
loss_component1

BokehModel(combine_events=True, render_bundle={'docs_json': {'639958f3-6574-409a-85a7-0a159ba2f790': {'defs': …

In [279]:
#Import dataframe
address = ("Data/District_Loss/heat_df.csv")
heat_df = pd.read_csv(address)
#Group the dataset by demographic group
heat_df= heat_df.set_index('group')

In [280]:
heat_df['3.0'] = heat_df['3.0']*-1
heat_df['4.0'] = heat_df['4.0']*-1
heat_df['5.0'] = heat_df['5.0']*-1
heat_df['6.0'] = heat_df['6.0']*-1
heat_df['7.0'] = heat_df['7.0']*-1
heat_df['8.0'] = heat_df['8.0']*-1
heat_df['3.0'] = heat_df['3.0']*100
heat_df['4.0'] = heat_df['4.0']*100
heat_df['5.0'] = heat_df['5.0']*100
heat_df['6.0'] = heat_df['6.0']*100
heat_df['7.0'] = heat_df['7.0']*100
heat_df['8.0'] = heat_df['8.0']*100
heat_df=heat_df.round(2)
heat_df.rename(columns={'3.0':'3rd',
                      '4.0':'4th',
                      '5.0':'5th',
                      '6.0':'6th',
                      '7.0':'7th',
                      '8.0':'8th'},inplace=True)

In [281]:
#Creates a heatmap based off of the STAAR/Percent Loss Data by demographic group and grade
subject_inputs = ['Math','Reading']
class LossDashboard(param.Parameterized):
    Subject = param.Selector(subject_inputs,default = subject_inputs[0])
    @param.depends('Subject')
    def lossplot(self):
        data = heat_df[heat_df['subject'] == self.Subject]
        data = data.drop(['subject'],axis=1)
        fig = px.imshow(data, labels=dict(x='Grade',y='Demographic Group',color='Learning Loss %'),text_auto='.1f',
        height=500,width=800,
        color_continuous_scale='balance_r',
       # labels={'Learning Loss':'Learning Loss '}
        range_color=(-4,4),
        aspect='none')
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=24,color='black'),
        title_x=.96,                
        margin={"r":10,"t":10,"l":0,"b":15})
        fig.update_coloraxes(cmid=0,colorbar_title_font_family='Times New Roman',colorbar_title_text='% Loss',
                             colorbar_title_font_size=24,colorbar_title_side='bottom',colorbar_thickness=10)
        return fig
    def panel(self):
        return pn.Column(
            pn.Param(self, width=350, name='Normalized Difference in STAAR Scores Between 2019 and 2021 By Demographic Group and Grade:',sizing_mode = 'fixed',background='#FFFFFF'),  
            self.lossplot
        )
dashboard = LossDashboard(name = 'Loss Dashboard')
loss_component2 = dashboard.panel()
loss_component2

BokehModel(combine_events=True, render_bundle={'docs_json': {'ec5be5cd-ff6c-4387-8d9f-c4fdbb80c31d': {'defs': …

# Texas District Level Maps

In [282]:
tx_map = gpd.read_file("Data/ISDmaps/Districts2020to2021.geojson")
label_data = pd.read_csv("Data/District_Loss/DATA_Texas_District_v1.csv")

In [283]:
label_data["NCES_DISTR"] = label_data["NCES_DISTR"].astype(int)
label_data = label_data.sort_values("NCES_DISTR")
tx_map = tx_map.sort_values("NCES_DISTR")
tx_map['NCES_DISTR'] = tx_map['NCES_DISTR'].astype(int)

In [284]:
merged_df = tx_map.merge(label_data, how = 'inner', on=['NCES_DISTR','DISTRICT_N'])

In [285]:
merged_df['Full-Time Equivalent(FTE) Teachers Difference (2019-2021)'] = (merged_df['FTE Teachers 2020-2021']-merged_df['FTE Teachers 2018-2019'])/merged_df['FTE Teachers 2018-2019']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2021)'] = (merged_df['Total Staff 2020-2021']-merged_df['Total Staff 2018-2019'])/merged_df['Total Staff 2018-2019']
merged_df['Total Number of Students Difference (2019-2021)'] = (merged_df['Total Students 2020-2021']-merged_df['Total Students 2018-2019'])/merged_df['Total Students 2018-2019']

merged_df['Full-Time Equivalent(FTE) Teachers Difference (2021-2022)'] = (merged_df['FTE Teachers 2021-2022']-merged_df['FTE Teachers 2020-2021'])/merged_df['FTE Teachers 2020-2021']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)'] = (merged_df['Total Staff 2021-2022']-merged_df['Total Staff 2020-2021'])/merged_df['Total Staff 2020-2021']
merged_df['Total Number of Students Difference (2021-2022)'] = (merged_df['Total Students 2021-2022']-merged_df['Total Students 2020-2021'])/merged_df['Total Students 2020-2021']

merged_df['Full-Time Equivalent(FTE) Teachers Difference (2019-2022)'] = (merged_df['FTE Teachers 2021-2022']-merged_df['FTE Teachers 2018-2019'])/merged_df['FTE Teachers 2018-2019']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)'] = (merged_df['Total Staff 2021-2022']-merged_df['Total Staff 2018-2019'])/merged_df['Total Staff 2018-2019']
merged_df['Total Number of Students Difference (2019-2022)'] = (merged_df['Total Students 2021-2022']-merged_df['Total Students 2018-2019'])/merged_df['Total Students 2018-2019']

In [286]:
merged_df = merged_df.round(2)

In [287]:
merged_df.rename(columns={'% LOSS Math 2019 to 2021':'Average Score Difference(%) in Math (2019-2021)',
                          '% LOSS Reading 2019 to 2021':'Average Score Difference(%) in Reading (2019-2021)',
                          '% LOSS All 2019 to 2021':'Average Score Difference(%) in All (2019-2021)',
                          'Label_Math (19-21)':'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2021)',
                          'Label_Reading (19-21)':'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)',
                          'Label_All (19-21)':'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)'},inplace=True)

merged_df.rename(columns={'% LOSS Math 2021 to 2022':'Average Score Difference(%) in Math (2021-2022)',
                          '% LOSS Reading 2021 to 2022':'Average Score Difference(%) in Reading (2021-2022)',
                          '% LOSS All 2021 to 2022':'Average Score Difference(%) in All (2021-2022)',
                          'Label_Math (21-22)':'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2021-2022)',
                          'Label_Reading (21-22)':'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)',
                          'Label_All (21-22)':'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)'},inplace=True)

merged_df.rename(columns={'% LOSS Math 2019 to 2022':'Average Score Difference(%) in Math (2019-2022)',
                          '% LOSS Reading 2019 to 2022':'Average Score Difference(%) in Reading (2019-2022)',
                          '% LOSS All 2019 to 2022':'Average Score Difference(%) in All (2019-2022)',
                          'Label_Math (19-22)':'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2022)',
                          'Label_Reading (19-22)':'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)',
                          'Label_All (19-22)':'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)'},inplace=True)

In [288]:
county_dict = pd.DataFrame()
county_dict['County #'] = comp_df['County #']
county_dict['County Name'] = comp_df['County Name']
cdict = pd.Series(county_dict['County #'].values,index=county_dict['County Name']).to_dict()
ndict = pd.Series(county_dict['County Name'].values,index=county_dict['County #']).to_dict()
label_data['County Number'] = label_data['County #']
label_data.replace({'County #': ndict}, inplace=True)
label_data.rename(columns = {'County #': 'County Name','County Number':'County #'}, inplace=True)

In [289]:
cdict

{'Anderson': 48001,
 'Andrews': 48003,
 'Angelina': 48005,
 'Aransas': 48007,
 'Archer': 48009,
 'Armstrong': 48011,
 'Atascosa': 48013,
 'Austin': 48015,
 'Bailey': 48017,
 'Bandera': 48019,
 'Bastrop': 48021,
 'Baylor': 48023,
 'Bee': 48025,
 'Bell': 48027,
 'Bexar': 48029,
 'Blanco': 48031,
 'Borden': 48033,
 'Bosque': 48035,
 'Bowie': 48037,
 'Brazoria': 48039,
 'Brazos': 48041,
 'Brewster': 48043,
 'Briscoe': 48045,
 'Brooks': 48047,
 'Brown': 48049,
 'Burleson': 48051,
 'Burnet': 48053,
 'Caldwell': 48055,
 'Calhoun': 48057,
 'Callahan': 48059,
 'Cameron': 48061,
 'Camp': 48063,
 'Carson': 48065,
 'Cass': 48067,
 'Castro': 48069,
 'Chambers': 48071,
 'Cherokee': 48073,
 'Childress': 48075,
 'Clay': 48077,
 'Cochran': 48079,
 'Coke': 48081,
 'Coleman': 48083,
 'Collin': 48085,
 'Collingsworth': 48087,
 'Colorado': 48089,
 'Comal': 48091,
 'Comanche': 48093,
 'Concho': 48095,
 'Cooke': 48097,
 'Coryell': 48099,
 'Cottle': 48101,
 'Crane': 48103,
 'Crockett': 48105,
 'Crosby': 48107

In [290]:
merged_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2019-2021)':'Number of Teachers (2019-2021)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2021)':'Number of Overall Staff (2019-2021)',
                          'Total Number of Students Difference (2019-2021)': 'Number of Students (2019-2021)',
                          'Average Score Difference(%) in Math (2019-2021)': 'STAAR Score in Math (2019-2021)',
                          'Average Score Difference(%) in Reading (2019-2021)': 'STAAR Score in Reading (2019-2021)',
                          'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss in Math (2019-2021)',
                          'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss in Reading (2019-2021)',
                          'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss (2019-2021)'},inplace=True)

merged_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2021-2022)':'Number of Teachers (2021-2022)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)':'Number of Overall Staff (2021-2022)',
                          'Total Number of Students Difference (2021-2022)': 'Number of Students (2021-2022)',
                          'Average Score Difference(%) in Math (2021-2022)': 'STAAR Score in Math (2021-2022)',
                          'Average Score Difference(%) in Reading (2021-2022)': 'STAAR Score in Reading (2021-2022)',
                          'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss in Math (2021-2022)',
                          'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss in Reading (2021-2022)',
                          'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss (2021-2022)'},inplace=True)

merged_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2019-2022)':'Number of Teachers (2019-2022)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)':'Number of Overall Staff (2019-2022)',
                          'Total Number of Students Difference (2019-2022)': 'Number of Students (2019-2022)',
                          'Average Score Difference(%) in Math (2019-2022)': 'STAAR Score in Math (2019-2022)',
                          'Average Score Difference(%) in Reading (2019-2022)': 'STAAR Score in Reading (2019-2022)',
                          'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss in Math (2019-2022)',
                          'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss in Reading (2019-2022)',
                          'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss (2019-2022)'},inplace=True)

In [291]:
merged_df['Number of Teachers (2019-2021)'] = merged_df['Number of Teachers (2019-2021)']*100
merged_df['Number of Overall Staff (2019-2021)'] = merged_df['Number of Overall Staff (2019-2021)']*100
merged_df['Number of Students (2019-2021)'] = merged_df['Number of Students (2019-2021)']*100
merged_df['STAAR Score in Math (2019-2021)'] = merged_df['STAAR Score in Math (2019-2021)']*100
merged_df['STAAR Score in Reading (2019-2021)'] = merged_df['STAAR Score in Reading (2019-2021)']*100
merged_df['Math% (2019-2021)'] = merged_df['Learning Loss in Math (2019-2021)']
merged_df['Reading% (2019-2021)'] = merged_df['Learning Loss in Reading (2019-2021)']
merged_df['Loss% (2019-2021)'] = merged_df['Learning Loss (2019-2021)']

merged_df['Number of Teachers (2021-2022)'] = merged_df['Number of Teachers (2021-2022)']*100
merged_df['Number of Overall Staff (2021-2022)'] = merged_df['Number of Overall Staff (2021-2022)']*100
merged_df['Number of Students (2021-2022)'] = merged_df['Number of Students (2021-2022)']*100
merged_df['STAAR Score in Math (2021-2022)'] = merged_df['STAAR Score in Math (2021-2022)']*100
merged_df['STAAR Score in Reading (2021-2022)'] = merged_df['STAAR Score in Reading (2021-2022)']*100
merged_df['Math% (2021-2022)'] = merged_df['Learning Loss in Math (2021-2022)']
merged_df['Reading% (2021-2022)'] = merged_df['Learning Loss in Reading (2021-2022)']
merged_df['Loss% (2021-2022)'] = merged_df['Learning Loss (2021-2022)']

merged_df['Number of Teachers (2019-2022)'] = merged_df['Number of Teachers (2019-2022)']*100
merged_df['Number of Overall Staff (2019-2022)'] = merged_df['Number of Overall Staff (2019-2022)']*100
merged_df['Number of Students (2019-2022)'] = merged_df['Number of Students (2019-2022)']*100
merged_df['STAAR Score in Math (2019-2022)'] = merged_df['STAAR Score in Math (2019-2022)']*100
merged_df['STAAR Score in Reading (2019-2022)'] = merged_df['STAAR Score in Reading (2019-2022)']*100
merged_df['Math% (2019-2022)'] = merged_df['Learning Loss in Math (2019-2022)']
merged_df['Reading% (2019-2022)'] = merged_df['Learning Loss in Reading (2019-2022)']
merged_df['Loss% (2019-2022)'] = merged_df['Learning Loss (2019-2022)']

In [292]:
def lossfuncmath(row):
    if row['Learning Loss in Math (2019-2021)']>0:
        val = 'Gain'
    elif row['Learning Loss in Math (2019-2021)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Math (2019-2021)'] = merged_df.apply(lossfuncmath, axis=1)

def lossfuncreading(row):
    if row['Learning Loss in Reading (2019-2021)']>0:
        val = 'Gain'
    elif row['Learning Loss in Reading (2019-2021)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Reading (2019-2021)'] = merged_df.apply(lossfuncreading, axis=1)

def lossfunc(row):
    if row['Learning Loss (2019-2021)']>0:
        val = 'Gain'
    elif row['Learning Loss (2019-2021)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Overall (2019-2021)'] = merged_df.apply(lossfunc, axis=1)


###############################################################


def lossfuncmath(row):
    if row['Learning Loss in Math (2021-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss in Math (2021-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Math (2021-2022)'] = merged_df.apply(lossfuncmath, axis=1)

def lossfuncreading(row):
    if row['Learning Loss in Reading (2021-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss in Reading (2021-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Reading (2021-2022)'] = merged_df.apply(lossfuncreading, axis=1)

def lossfunc(row):
    if row['Learning Loss (2021-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss (2021-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Overall (2021-2022)'] = merged_df.apply(lossfunc, axis=1)


###############################################################


def lossfuncmath(row):
    if row['Learning Loss in Math (2019-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss in Math (2019-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Math (2019-2022)'] = merged_df.apply(lossfuncmath, axis=1)

def lossfuncreading(row):
    if row['Learning Loss in Reading (2019-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss in Reading (2019-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Reading (2019-2022)'] = merged_df.apply(lossfuncreading, axis=1)

def lossfunc(row):
    if row['Learning Loss (2019-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss (2019-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Overall (2019-2022)'] = merged_df.apply(lossfunc, axis=1)

In [293]:
merged_df.to_csv('Data/DistrictLevelstats.csv')

In [294]:
#merged_df['Number of Overall Staff'] = merged_df['Number of Overall Staff'].astype(int)
#merged_df['Number of Students'] = merged_df['Number of Students'].astype(int)
#merged_df['Number of Teachers'] = merged_df['Number of Teachers'].astype(int)
#merged_df['STAAR Score in Math'] = merged_df['STAAR Score in Math'].astype(int)
#merged_df['STAAR Score in Reading'] = merged_df['STAAR Score in Reading'].astype(int)

merged_df['Number of Overall Staff (2019-2021)'] = merged_df['Number of Overall Staff (2019-2021)'].astype(int)
merged_df['Number of Students (2019-2021)'] = merged_df['Number of Students (2019-2021)'].astype(int)
merged_df['Number of Teachers (2019-2021)'] = merged_df['Number of Teachers (2019-2021)'].astype(int)
merged_df['STAAR Score in Math (2019-2021)'] = merged_df['STAAR Score in Math (2019-2021)'].astype(int)
merged_df['STAAR Score in Reading (2019-2021)'] = merged_df['STAAR Score in Reading (2019-2021)'].astype(int)

merged_df['Number of Overall Staff (2021-2022)'] = merged_df['Number of Overall Staff (2021-2022)'].astype(int)
merged_df['Number of Students (2021-2022)'] = merged_df['Number of Students (2021-2022)'].astype(int)
merged_df['Number of Teachers (2021-2022)'] = merged_df['Number of Teachers (2021-2022)'].astype(int)
merged_df['STAAR Score in Math (2021-2022)'] = merged_df['STAAR Score in Math (2021-2022)'].astype(int)
merged_df['STAAR Score in Reading (2021-2022)'] = merged_df['STAAR Score in Reading (2021-2022)'].astype(int)

merged_df['Number of Overall Staff (2019-2022)'] = merged_df['Number of Overall Staff (2019-2022)'].astype(int)
merged_df['Number of Students (2019-2022)'] = merged_df['Number of Students (2019-2022)'].astype(int)
merged_df['Number of Teachers (2019-2022)'] = merged_df['Number of Teachers (2019-2022)'].astype(int)
merged_df['STAAR Score in Math (2019-2022)'] = merged_df['STAAR Score in Math (2019-2022)'].astype(int)
merged_df['STAAR Score in Reading (2019-2022)'] = merged_df['STAAR Score in Reading (2019-2022)'].astype(int)

In [295]:
df_1921 = merged_df[['NAME','County #','DISTRICT_N','Number of Teachers (2019-2021)',
                    'Number of Overall Staff (2019-2021)',
                    'Number of Students (2019-2021)',
                    'STAAR Score in Math (2019-2021)',
                    'STAAR Score in Reading (2019-2021)',
                    'Math% (2019-2021)',
                    'Reading% (2019-2021)',
                    'Loss% (2019-2021)']]

df_2122 = merged_df[['NAME','County #','DISTRICT_N','Number of Teachers (2021-2022)',
                    'Number of Overall Staff (2021-2022)',
                    'Number of Students (2021-2022)',
                    'STAAR Score in Math (2021-2022)',
                    'STAAR Score in Reading (2021-2022)',
                    'Math% (2021-2022)',
                    'Reading% (2021-2022)',
                    'Loss% (2021-2022)']]

df_1922 = merged_df[['NAME','County #','DISTRICT_N','Number of Teachers (2019-2022)',
                    'Number of Overall Staff (2019-2022)',
                    'Number of Students (2019-2022)',
                    'STAAR Score in Math (2019-2022)',
                    'STAAR Score in Reading (2019-2022)',
                    'Math% (2019-2022)',
                    'Reading% (2019-2022)',
                    'Loss% (2019-2022)']]

In [296]:
df_1921.rename(columns = {'Number of Teachers (2019-2021)':'Number of Teachers',
                           'Number of Overall Staff (2019-2021)':'Number of Staff',
                           'Number of Students (2019-2021)':'Number of Students',
                           'STAAR Score in Math (2019-2021)':'STAAR Score in Math',
                           'STAAR Score in Reading (2019-2021)':'STAAR Score in Reading',
                           'Math% (2019-2021)':'Math%',
                           'Reading% (2019-2021)':'Reading%',
                           'Loss% (2019-2021)':'Loss%'}, inplace=True)

df_2122.rename(columns = {'Number of Teachers (2021-2022)':'Number of Teachers',
                           'Number of Overall Staff (2021-2022)':'Number of Staff',
                           'Number of Students (2021-2022)':'Number of Students',
                           'STAAR Score in Math (2021-2022)':'STAAR Score in Math',
                           'STAAR Score in Reading (2021-2022)':'STAAR Score in Reading',
                           'Math% (2021-2022)':'Math%',
                           'Reading% (2021-2022)':'Reading%',
                           'Loss% (2021-2022)':'Loss%'}, inplace=True)

df_1922.rename(columns = {'Number of Teachers (2019-2022)':'Number of Teachers',
                           'Number of Overall Staff (2019-2022)':'Number of Staff',
                           'Number of Students (2019-2022)':'Number of Students',
                           'STAAR Score in Math (2019-2022)':'STAAR Score in Math',
                           'STAAR Score in Reading (2019-2022)':'STAAR Score in Reading',
                           'Math% (2019-2022)':'Math%',
                           'Reading% (2019-2022)':'Reading%',
                           'Loss% (2019-2022)':'Loss%'}, inplace=True)

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\4287373170.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\4287373170.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Daniel Payan\AppData\Local\Temp\ipykernel_2440\4287373170.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [298]:
merged_df['Total Funding'] = merged_df['CARES ESSER I 20'] + merged_df['CRRSA ESSER II 21'] + merged_df['ARP ESSER III 21'] + merged_df['ESSER-SUPP 22'] + merged_df['ESSER-SUPP 23']
merged_df['Funding Per Student'] = merged_df['Total Funding']/merged_df['Number of Students (2021-2022)']

In [299]:
#Creates a county map of Texas based off of the National Center on Education Statistics DataFrame
input_list = ['STAAR Score in Math',
              'STAAR Score in Reading',
              'Number of Students',
              'Number of Teachers',
              'Number of Staff'
             ]
funding_list = ['CARES ESSER I 20',
              'CRRSA ESSER II 21',
              'ARP ESSER III 21',
              'ESSER-SUPP 22',
              'ESSER-SUPP 23',
              'Funding Per Student'
              'Total Funding']
year_list = ['2019-2021','2021-2022','2019-2022','Funding']
district_list = label_data['County Name'].values
district_list = sorted(district_list)
class MapDashboard(param.Parameterized):
    

    __   = param.Selector(default='Hays',objects=district_list,check_on_set=False)
    ___  = param.Selector(default=year_list[0],objects=year_list)
    _    = param.ObjectSelector(default=input_list[0],objects=input_list)
    lists = {'2019-2021':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             '2021-2022':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             '2019-2022':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             'Funding':['CARES ESSER I 20','CRRSA ESSER II 21','ARP ESSER III 21','ESSER-SUPP 22','ESSER-SUPP 23','Funding Per Student','Total Funding']
            }
    @param.depends('___',watch=True)
    def update_list(self):
        up_list = self.lists[self.___]
        self.param['_'].objects = up_list
        self._ = up_list[0]
        
    def get_map(self):
        county_num = cdict[self.__]
        if self.___ == '2019-2021':
            yr_df = df_1921.loc[df_1921['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == '2021-2022':
            yr_df = df_2122.loc[df_2122['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == '2019-2022':
            yr_df = df_1922.loc[df_1922['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == 'Funding':
            yr_df = merged_df.loc[merged_df['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',self._]]
        
        dis_df = merged_df.loc[merged_df['County #'] == county_num]
        
        if self.___ == 'Funding':
            minimum = 0
            maximum = 10000000
            fig = px.choropleth(data, geojson=dis_df, locations='DISTRICT_N', color= self._,
            color_continuous_scale='balance_r',
            range_color=(minimum,maximum),
            featureidkey='properties.DISTRICT_N',
            title =self.__ + ' County',
            custom_data = [data['NAME'],self._,],
            height=550, width=480)
            fig.update_layout(coloraxis_colorbar=dict(tickvals=[1000000,3000000,5000000,7000000,9000000],ticktext=['0M','2M','4M','6M','8M']))

        else:
            minimum = -10
            maximum = 10
            fig = px.choropleth(data, geojson=dis_df, locations='DISTRICT_N', color= self._,
            color_continuous_scale='balance_r',
            range_color=(minimum,maximum),
            featureidkey='properties.DISTRICT_N',
            title =self.__ + ' County',
            custom_data = [data['NAME'],self._,data['Math%'],data['Reading%'],data['Loss%']],
            height=550, width=480
                               )
            fig.update_layout(coloraxis_colorbar=dict(tickvals=[-8,-4,0,4,8],ticktext=['-10','-5','0','5','10']))
            
        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-0.1,colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=24,colorbar_title_side='bottom',colorbar_thickness=4)

        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-0.1,colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=24,colorbar_title_side='bottom',colorbar_thickness=4)
        if self.___ == 'Funding':
            if self._ == 'Total Funding':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '$%{customdata[1]} in ' + self._
                #fig.update_traces(hovertemplate=hovertemp)
            else:
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '$%{customdata[1]} in ' + self._
        else:
            hovertemp = ''
            hovertemp += '%{customdata[0]}<br>'
            hovertemp += '%{customdata[1]}% change in ' + self._
        
            
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations',visible=False)
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',size=24,color='black'),
        title_x=.5,
        title_y=.97,                  
        margin={"r":10,"t":0,"l":0,"b":0})
        return fig
    
    def get_description(self):
        if self.___ == "2019-2021":
            if self._ == 'STAAR Score in Math':
                description = pn.pane.HTML('The percent difference in Math STAAR scores between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'STAAR Score in Reading':
                description = pn.pane.HTML('The percent difference in Reading STAAR scores between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Teachers':
                description = pn.pane.HTML('The percent difference in Full-Time Teachers between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Staff':
                description = pn.pane.HTML('The percent difference in staff between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._== 'Number of Students':
                description = pn.pane.HTML('The percent difference in students between 2019 and 2021 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
                
        elif self.___ == '2021-2022':
            if self._ == 'STAAR Score in Math':
                description = pn.pane.HTML('The percent difference in Math STAAR scores between 2021 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'STAAR Score in Reading':
                description = pn.pane.HTML('The percent difference in Reading STAAR scores between 2021 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Teachers':
                description = pn.pane.HTML('The percent difference in Full-Time Teachers between 2021 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Staff':
                description = pn.pane.HTML('The percent difference in staff between 2021 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._== 'Number of Students':
                description = pn.pane.HTML('The percent difference in students between 2021 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
                
        elif self.___ == '2019-2022':
            if self._ == 'STAAR Score in Math':
                description = pn.pane.HTML('The percent difference in Math STAAR scores between 2019 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'STAAR Score in Reading':
                description = pn.pane.HTML('The percent difference in Reading STAAR scores between 2019 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Teachers':
                description = pn.pane.HTML('The percent difference in Full-Time Teachers between 2019 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._ == 'Number of Staff':
                description = pn.pane.HTML('The percent difference in staff between 2019 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
            elif self._== 'Number of Students':
                description = pn.pane.HTML('The percent difference in students between 2019 and 2022 for a district.',
                            style={'color':'black','font-size':'20px','width':'125%',
                                  'text-align':'left',
                                  'font-family':'Times New Roman'})
        elif self.___ == 'Funding':
            if self._ == 'CARES ESSER I 20':
                description = pn.pane.HTML('The dollar amount of funding granted to a district through the CARES ESSER I 20 Program.',
                           style={'color':'black','font-size':'20px','width':'125%',
                          'text-align':'left',
                          'font-family':'Times New Roman'})
            elif self._ == 'CRRSA ESSER II 21':
                description = pn.pane.HTML('The dollar amount of funding granted to a district through the CRRSA ESSER II 21 Program.',
                           style={'color':'black','font-size':'20px','width':'125%',
                          'text-align':'left',
                          'font-family':'Times New Roman'})      
            elif self._ == 'ARP ESSER III 21':
                description = pn.pane.HTML('The dollar amount of funding granted to a district through the ARP ESSER III 21 Program.',
                           style={'color':'black','font-size':'20px','width':'125%',
                          'text-align':'left',
                          'font-family':'Times New Roman'})      
            elif self._ == 'ESSER-SUPP 22':
                description = pn.pane.HTML('The dollar amount of funding granted to a district through the ESSER-SUPP 22 Program.',
                           style={'color':'black','font-size':'20px','width':'125%',
                          'text-align':'left',
                          'font-family':'Times New Roman'})      
            elif self._ == 'ESSER-SUPP 23':
                description = pn.pane.HTML('The dollar amount of funding granted to a district through the ESSER-SUPP 23 Program.',
                           style={'color':'black','font-size':'20px','width':'125%',
                          'text-align':'left',
                          'font-family':'Times New Roman'})      
            elif self._ == 'Funding Per Student':
                description = pn.pane.HTML('The total dollar amount of funding per student granted to a district.',
                           style={'color':'black','font-size':'20px','width':'125%',
                          'text-align':'left',
                          'font-family':'Times New Roman'})
            elif self._ == 'Total Funding':
                description = pn.pane.HTML('The total dollar amount of funding granted to a district.',
                           style={'color':'black','font-size':'20px','width':'125%',
                          'text-align':'left',
                          'font-family':'Times New Roman'}) 
        return description
    
    def panel(self):
        return pn.Row(
            pn.Column(
            pn.pane.HTML("""<h1>Select A County & Variable To View At The District Level </h1>""",
                              style={'color':'black','font-size':'14px','width':'100%',
                                     'text-align':'center',
                                     'font-family':'Times New Roman'}),
            pn.Param(self,width=550,background='#FFFFFF',
                     widgets={'__':{'widget_type':pn.widgets.AutocompleteInput,'placeholder':'Enter a county name...'},
                             '___':{'widget_type':pn.widgets.RadioButtonGroup},}
                    ),
         
            pn.Spacer(height=50),
            pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            style={'color':'black','font-size':'14px','width':'100%',
                                  'text-align':'center',
                                  'font-family':'Times New Roman'}),
            pn.Spacer(height= 5),
            self.get_description
            #pn.Param(self, width=500,background='#FFFFFF'),
            ),
            pn.Spacer(width=150),
            self.get_map
        )
dashboard = MapDashboard(name='')
map_component2 = dashboard.panel()
map_component2

Launching server at http://localhost:18133


# County Level Dashboard

In [300]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=171,width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png",height=140,width=350)


title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)

content_bar = pn.Row(pn.Spacer(width=225),map_component1)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=6, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
gspec[1:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:18134


# District Level Dashboard

In [301]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=171,width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png",height=140,width=350)



title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)

content_bar = pn.Row(pn.Spacer(width=250),map_component2)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=6, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
gspec[1:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:18135


# Heatmap & Histogram Dashboard

In [303]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=171,width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png",height=140,width=350)



title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)

content_bar = pn.Row(pn.Spacer(width=100),loss_component1, pn.Spacer(width=80),loss_component2)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=6, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
gspec[2:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:18137


# Definitions

In [304]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
#datalab_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/dataimg.png",height=220,width=410)
#txst_img = pn.pane.PNG("C:/Users/Daniel Payan/Downloads/txst1.png",width=410, height=220)
datalab_img = pn.pane.PNG("Data/Images/datalabimage.png",height=171,width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png",height=140,width=350)



title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)
index_pane = pn.pane.HTML("""<h1>Variable Definitions</h1>""", 
                          style={'color':'white','width':'168%','font-size':'24px','text-align':'center',    
                      #    'outline-style':'solid','outline-color':'#30231D',   
                      #    'outline-width':'thick',
                                 'background-color':'#501214',
                          'font-family':'Times New Roman',
                          'border':'10px solid black'})
table = pn.pane.HTML(
"""
<table style="width:105%">
<th style = 'font-size':'8px'>
  <tr>
    <th>Variable</th>
    <th>Reflects the percent difference of...</th>
  </tr>
  <tr>
    <td>STAAR Score in Math</td>
    <td>Math STAAR scores between X and Y years for a district/county.</td>
  </tr>
  <tr>
    <td>STAAR Score in Reading</td>
    <td>Reading STAAR scores between X and Y years for a district/county.</td>
  </tr>
  <tr>
    <td>Labor Force</td>
    <td>Labor force between X and Y years for a county.</td>
 </tr>
 <tr>
    <td>Number of Overall Staff</td>
    <td>Staff between X and Y years for a district/county.</td>
 </tr>
 <tr>
    <td>Number of Students</td>
    <td>Students between X and Y years for a district/county.</td>
 </tr>
 <tr>
    <td>Number of Teachers</td>
    <td>Full-Time Teachers between X and Y years for a district/county.</td>
 </tr>
</table>
""", style={'width':'168%','font-size':'32px','text-align':'center',    
            #'outline-style':'solid','outline-color':'#30231D',   
           # 'outline-width':'thick',
            'font-family':'Times New Roman',
            'border':'4px solid black'})
content_bar = pn.Row(pn.Spacer(width=145),
                     pn.Column(index_pane,table))

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=12, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
#gspec[1:6, 0] = pn.Spacer()
#gspec[1:6, 5] = pn.Spacer()
gspec[2:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:18138
